# A Beginners Guide to Federated Learning

Follow steps in https://towardsdatascience.com/federated-learning-a-step-by-step-implementation-in-tensorflow-aac568283399

## Setting Up the Environment and Dependencies

In [34]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

from FL_initial_test_util import *

In [35]:
#declear path to your mnist data folder
img_path = 'data/MNIST/trainingSet/trainingSet'

#get the path list using the path object
image_paths = list(paths.list_images(img_path))

#apply our function
image_list, label_list = load(image_paths, verbose=10000)

#binarize the labels
lb = LabelBinarizer()
label_list = lb.fit_transform(label_list)

#split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(image_list, 
                                                    label_list, 
                                                    test_size=0.1, 
                                                    random_state=42)

[INFO] processed 10000/42000
[INFO] processed 20000/42000
[INFO] processed 30000/42000
[INFO] processed 40000/42000


In [36]:
#create clients
clients = create_clients(X_train, y_train, num_clients=10, initial='client')

In [37]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [39]:
lr = 0.01 
comms_round = 100
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = tf.keras.optimizers.legacy.SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               ) 

/Users/lubingguang/miniconda3/envs/tensorflow/lib/python3.8/site-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [40]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(784, 10)
        
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(784, 10)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

132/132 [==============================] - 0s 1ms/step
comm_round: 0 | global_acc: 87.952% | global_loss: 1.669370174407959
132/132 [==============================] - 0s 1ms/step
comm_round: 1 | global_acc: 90.262% | global_loss: 1.6220005750656128
132/132 [==============================] - 0s 1ms/step
comm_round: 2 | global_acc: 91.214% | global_loss: 1.602520227432251
132/132 [==============================] - 0s 2ms/step
comm_round: 3 | global_acc: 92.238% | global_loss: 1.5920063257217407
132/132 [==============================] - 0s 1ms/step
comm_round: 4 | global_acc: 92.595% | global_loss: 1.5834453105926514
132/132 [==============================] - 0s 1ms/step
comm_round: 5 | global_acc: 92.929% | global_loss: 1.5755285024642944
132/132 [==============================] - 0s 1ms/step
comm_round: 6 | global_acc: 93.524% | global_loss: 1.5689549446105957
132/132 [==============================] - 0s 1ms/step
comm_round: 7 | global_acc: 93.619% | global_loss: 1.5646910667419434
13